<a href="https://colab.research.google.com/github/FranziskoB/Regressao_simples_multipla/blob/main/03_Regress%C3%A3o_linear_multipla_Dummies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MODELAGEM DE UMA REGRESSÃO LINEAR MULTIPLA COM DUMMIES

## Bibliotecas

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import seaborn as sns 
import matplotlib.pyplot as plt
import numpy as np

## Importando os dados

In [ ]:
corrupcao = pd.read_csv('https://github.com/FranziskoB/Regressao_simples_multipla/raw/main/dados/corrupcao.csv')

In [ ]:
corrupcao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   pais    50 non-null     object 
 1   cpi     50 non-null     float64
 2   regiao  50 non-null     object 
dtypes: float64(1), object(2)
memory usage: 1.3+ KB


In [ ]:
corrupcao.head(5)

,pais,cpi,regiao
0,Argentina,3.9,América do Sul
1,Australia,8.7,Oceania
2,Austria,7.9,Europa
3,Belgium,7.1,Europa
4,Brazil,4.0,América do Sul


In [ ]:
corrupcao.describe()

,cpi
count,50.000000
mean,4.894000
std,2.678974
min,0.800000
25%,2.575000
50%,3.950000
75%,7.475000
max,9.300000


## Obtendo dummies das variáveis qualitativas

In [ ]:
regiao_dummies=pd.get_dummies(corrupcao[['regiao']],drop_first=True) # Criando as colunas dummies

In [ ]:
regiao_dummies.head()

,regiao_EUA e Canadá,regiao_Europa,regiao_Oceania,regiao_Ásia
0,0,0,0,0
1,0,0,1,0
2,0,1,0,0
3,0,1,0,0
4,0,0,0,0


In [ ]:
corrupcao_dummies=pd.merge(corrupcao, regiao_dummies, left_index=True, right_index=True)#Juntando os dois dataframes

## ESTIMAÇÃO DO MODELO DE REGRESSÃO

In [ ]:
# é necessário adicionar uma constante a matriz X
X_sm = sm.add_constant(corrupcao_dummies[['regiao_EUA e Canadá','regiao_Europa','regiao_Oceania','regiao_Ásia']])
Y_sm=corrupcao_dummies[['cpi']]
# OLS vem de Ordinary Least Squares e o método fit irá treinar o modelo
results = sm.OLS(Y_sm, X_sm).fit()


/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [ ]:
# mostrando as estatísticas do modelo
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    cpi   R-squared:                       0.603
Model:                            OLS   Adj. R-squared:                  0.568
Method:                 Least Squares   F-statistic:                     17.09
Date:                Sat, 25 Feb 2023   Prob (F-statistic):           1.37e-08
Time:                        18:28:26   Log-Likelihood:                -96.620
No. Observations:                  50   AIC:                             203.2
Df Residuals:                      45   BIC:                             212.8
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   4.1800      0.788      5.306      0.000       2.593       5.767
regiao_EUA e Canadá     3.8200      1.474      2.592      0.013       0.852       6.788
regiao_Europa           2.0783      0.866      2.400      0.021       0.334       3.822
regiao_Oceania          4.8200      1.474      3.271      0.002       1.852       7.788
regiao_Ásia            -1.8506      0.896     -2.065      0.045      -3.655      -0.046
==============================================================================
Omnibus:                        1.205   Durbin-Watson:                   1.928
Prob(Omnibus):                  0.547   Jarque-Bera (JB):                1.083
Skew:                          -0.347   Prob(JB):                        0.582
Kurtosis:                       2.804   Cond. No.                         9.35
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

O modelo apresentou R² = 	0.603, intercepto e coefieficientes estatisticamente significativos para o nível de significancia de 95%. 

In [ ]:
# mostrando as previsões para o mesmo conjunto passado
corrupcao_dummies['yhat']=results.predict(X_sm)
corrupcao_dummies['erro']=corrupcao_dummies['yhat']-corrupcao_dummies['cpi']

In [ ]:
print(results.params.const)

4.179999999999999


In [ ]:
corrupcao_dummies.head(5)

,pais,cpi,regiao,regiao_EUA e Canadá,regiao_Europa,regiao_Oceania,regiao_Ásia,yhat,erro
0,Argentina,3.9,América do Sul,0,0,0,0,4.180000,0.280000
1,Australia,8.7,Oceania,0,0,1,0,9.000000,0.300000
2,Austria,7.9,Europa,0,1,0,0,6.258333,-1.641667
3,Belgium,7.1,Europa,0,1,0,0,6.258333,-0.841667
4,Brazil,4.0,América do Sul,0,0,0,0,4.180000,0.180000


## gráfico

In [ ]:

import plotly.express as px
corrupcao_dummies=corrupcao_dummies.sort_values(by='yhat')

yhat_regiao=corrupcao_dummies.groupby('regiao')[['yhat']].mean().reset_index().sort_values(by='yhat')
x_pred=yhat_regiao['regiao'].values
y_pred=yhat_regiao['yhat']
fig =px.line(yhat_regiao,x="regiao", y="yhat", color_discrete_sequence=["red"])
fig.add_traces(px.scatter(corrupcao_dummies, x="regiao", y="cpi", text="pais", log_x=False, size_max=0, color_discrete_sequence=["yellow"]).data)
fig.update_traces(textposition='top right', textfont_size=10)